In [162]:
from math import log
from collections import Counter

In [ ]:
def fmm_word_seg(sentence, lexicon, max_word_len):
    begin = 0
    end = min(begin + max_word_len, len(sentence))
    words = []
    while begin < end:
        word = sentence[begin:end]
        if word in lexicon or end - begin == 1:
            words.append(word)
            begin = end
            end = min(begin + max_word_len, len(sentence))
        else:
            end -= 1

    return words


def rmm_word_seg(sentence, lexicon, max_word_len):
    begin = len(sentence)
    end = max(begin - max_word_len, 0)
    words = []
    while begin > end:
        word = sentence[end:begin]
        if word in lexicon or begin - end == 1:
            words.append(word)
            begin = end
            end = max(begin - max_word_len, 0)
        else:
            end += 1

    return words[::-1]


def bmm_word_seg(sentence, lexicon, max_word_len):
    fmm_words = fmm_word_seg(sentence, lexicon, max_word_len)
    rmm_words = rmm_word_seg(sentence, lexicon, max_word_len)

    if fmm_words == rmm_words:
        return fmm_words

    if len(fmm_words) < len(rmm_words):
        return fmm_words
    elif len(fmm_words) > len(rmm_words):
        return rmm_words

    fmm_single_words = 0
    rmm_single_words = 0
    for i in fmm_words:
        if len(i) == 1:
            fmm_single_words += 1
    for i in rmm_words:
        if len(i) == 1:
            rmm_single_words += 1

    if fmm_single_words < rmm_single_words:
        return fmm_words
    elif fmm_single_words > rmm_single_words:
        return rmm_words
    
    fmm_not_lexicon_words = 0
    rmm_not_lexicon_words = 0

    for i in fmm_words:
        if i not in lexicon:
            fmm_not_lexicon_words += 1

    for i in rmm_words:
        if i not in lexicon:
            rmm_not_lexicon_words += 1

    if fmm_not_lexicon_words < rmm_not_lexicon_words:
        return fmm_words
    else:
        return rmm_words 


In [164]:
def load_lexicon(filename):
    lexicon = []
    words_count = []
    max_word_len = 0

    with open(filename) as file:
        for line in file:
            lexicon.append(line.strip().split()[0])
            words_count.append(int(line.strip().split()[1]))
            if len(line.strip()) > max_word_len:
                max_word_len = len(line.strip())
    
    return lexicon, words_count, max_word_len

In [165]:
lexicon, words_count, max_word_len = load_lexicon('./dict.txt.big')

sentence = '南京市长江大桥'
print(fmm_word_seg(sentence, lexicon, max_word_len))
print(rmm_word_seg(sentence, lexicon, max_word_len))
print(dmm_word_seg(sentence, lexicon, max_word_len))

['南京市', '长江大桥']
['南京市', '长江大桥']
['南京市', '长江大桥']


In [166]:
from jieba.finalseg.prob_start import P as prob_start
from jieba.finalseg.prob_trans import P as prob_trans
#from jieba.finalseg.prob_emit import P as prob_emit


print(prob_start, prob_trans, sep='\n')

{'B': -0.26268660809250016, 'E': -3.14e+100, 'M': -3.14e+100, 'S': -1.4652633398537678}
{'B': {'E': -0.51082562376599, 'M': -0.916290731874155}, 'E': {'B': -0.5897149736854513, 'S': -0.8085250474669937}, 'M': {'E': -0.33344856811948514, 'M': -1.2603623820268226}, 'S': {'B': -0.7211965654669841, 'S': -0.6658631448798212}}


In [169]:
count_emit = {label:Counter() for label in "SBME"}
for i in range(len(words_count)):
    if len(lexicon[i]) == 1:
        count_emit['S'][lexicon[i]] += words_count[i]
    else:
        count_emit['B'][lexicon[i][0]] += words_count[i]
        count_emit['E'][lexicon[i][-1]] += words_count[i]
        for char in lexicon[i][1:-1]:
            count_emit['M'][char] += words_count[i]
log_total = {label:log(sum(count_emit[label].values())) for label in "SBME"}
prob_emit = {label:{char:log(count_emit[label][char]) - log_total[label] for char in count_emit[label]} for label in "SBME"}

In [170]:
MIN_FLOAT = -3.14e100
PrevStatus = {
    'B': 'ES',
    'M': 'MB',
    'S': 'SE',
    'E': 'BS'
}

def viterbi(sentence, states, prob_start, prob_trans, prob_emit):
    V = [{}]
    path = {}

    for state in states:
        V[0][state] = prob_start[state] + prob_emit[state].get(sentence[0], MIN_FLOAT)
        path[state] = [state]

    for i in range(1,len(sentence)):
        V.append({})
        newpath = {}

        for state in states:
            prob, _state = max([V[i-1][st] + prob_trans[st].get(state, MIN_FLOAT) + prob_emit[state].get(sentence[i], MIN_FLOAT), st] for st in PrevStatus[state])
            V[i][state] = prob
            newpath[state] = path[_state] + [state]

        path = newpath

    prob, state = max((V[len(sentence) - 1][state], state) for state in 'ES')
    return prob, path[state]

def HMM_seg(sentence):
    prob, state = viterbi(sentence, 'SBME', prob_start, prob_trans, prob_emit)

    print(prob, state)

    words = []

    for i in range(len(sentence)):
        if state[i] in 'BS':
            words.append(sentence[i])
        else:
            words[-1] += sentence[i]

    return words

In [175]:
sentence = '单字词的概率好大'

print(HMM_seg(sentence))

-59.462186648667654 ['B', 'E', 'S', 'S', 'B', 'E', 'S', 'S']
['单字', '词', '的', '概率', '好', '大']
